# Credit Card Transaction Data Cleanup and Prep 

This notebook shows the steps for cleanup and preparing the credit card transaction data for follow on GNN training with GraphSAGE.

### The dataset:
 * 'Generate Fake Credit Card Transaction [Data](https://www.kaggle.com/datasets/kartik2112/fraud-detection), Including Fraudulent Transactions' using https://github.com/namebrandon/Sparkov_Data_Generation
 * Released under CC0: Public Domain

Contains 1,296,675 records with 15 fields, one field being the "is fraud" label which we use for training.

### Goals
The goal is to:
 * Understand and transform the data
   * Correlation analysis to select important predictors 
   * Encode categorical fields
   * Scale numerical columns
   * Create a continuous node index across users, merchants, and transactions
     * having node ID start at zero and then be contiguous is critical for creation of Compressed Sparse Row (CSR) formatted data without wasting memory.
 * Produce:
   * For XGBoost:
     * Training   - all transactions in 2019
     * Validation - all transactions between January and May in 2020
     * Test.      - all transactions after May 2020
   * For GNN
     * Training Data 
       * Edge List 
       * Feature data
   * Test set - all transactions after May 2020



### Graph formation
Given that we are limited to just the data in the transaction file, the ideal model would be to have a bipartite graph of Users to Merchants where the edges represent the credit card transaction and then perform Link Classification on the Edges to identify fraud. Unfortunately the current version of cuGraph does not support GNN Link Prediction. That limitation will be lifted over the next few release at which time this code will be updated. Luckily, there is precedence for viewing transactions as nodes and then doing node classification using the popular GraphSAGE GNN. That is the approach this code takes. The produced graph will be a tri-partite graph where each transaction is represented as a node.

<img src="../img/3-partite.jpg" width="35%"/>


### Features
For the XGBoost approach, there is no need to generate empty features for the Merchants. However, for GNN processing, every node needs to have the same set of feature data. Therefore, we need to generate empty features for the User and Merchant nodes. 

-----

#### Import the necessary libraries.  In this case will be use cuDF and perform most of the data prep in GPU


In [1]:
import json
import math
import os
import pickle

import cudf
import numpy as np
import pandas as pd
import scipy.stats as ss
from category_encoders import BinaryEncoder
from scipy.stats import gaussian_kde, pointbiserialr
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler


-------
#### Define some arguments

In [2]:
# Whether the graph is undirected
make_undirected = True

# Whether to spread features across Users and Merchants nodes
spread_features = False

# Whether we should under-sample majority class (i.e. non-fraud transactions)
under_sample = True

# Ration of fraud and non-fraud transactions in case we under-sample the majority class
fraud_ratio = 0.1


In [3]:
sparkov_base_path = '../data/Sparkov'
sparkov_raw_file_path = os.path.join(sparkov_base_path, 'raw', 'fraudTrain.csv')
sparkov_xgb = os.path.join(sparkov_base_path, 'xgb')
sparkov_gnn = os.path.join(sparkov_base_path, 'gnn')
if not os.path.exists(sparkov_xgb):
    os.makedirs(sparkov_xgb)
if not os.path.exists(sparkov_gnn):
    os.makedirs(sparkov_gnn)

--------
## Load and understand the data

In [4]:
# Read the dataset
data = cudf.read_csv(sparkov_raw_file_path, index_col=0)

In [ ]:
# optional - take a look at the data 
data.head(5)

### Findings
* Nominal categorical fields - 'cc_num', 'merchant', 'category', 'first', 'last', 'street', 'city', 'state', 'zip', 'job', 'trans_num'
* Numerical fields - 'amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long'
* Timestamp fields - 'dob', 'trans_date_trans_time', 'unix_time'
* Target label - 'is_fraud'


#### How many transactions are fraud?

In [ ]:
data['is_fraud'].value_counts()

In [ ]:
# Percentage of fraud transactions
100.0*(data['is_fraud'] == 1).sum()/len(data)

##### Findings - The dataset is extremely imbalanced, only 0.58% of the transactions are fraud

#### Check if are there Null values in the data

In [ ]:
# Check if any column has missing values
data.isnull().sum()


###### Great, none of the columns have null values

##### Save a few transactions before any operations on data

In [9]:
# Write a few raw transactions for model's inference notebook
out_path = os.path.join(sparkov_xgb, 'example_transactions.csv')
data.tail(10).to_pandas().to_csv(out_path, header=True, index=False)

#### Convert 'dob' to 'age' w.r.t. a reference date

In [10]:
data['dob'] = cudf.to_datetime(data['dob'])

In [11]:

one_nanosecond = np.timedelta64(1, 'ns')
nanoseconds_in_year = 365.25 * 24 * 60 * 60 * 1e9
reference_date =  cudf.to_datetime('2024-10-30') 

In [12]:
data['age'] = data['dob'].apply(lambda dob: (reference_date - dob)/ one_nanosecond / nanoseconds_in_year )

#### Split transaction time in year, month, day and time where time indicate number of minutes

In [13]:
tx_date_time =  cudf.to_datetime(data.trans_date_trans_time)

In [14]:
data['year'] = tx_date_time.dt.year
data['month'] = tx_date_time.dt.month
data['day'] = tx_date_time.dt.day
data['time'] = tx_date_time.dt.hour*60 +  tx_date_time.dt.minute


##### Observations

* we can treat 'year', 'month', 'day' as ordinal fields and time as numerical field

### From ('lat', 'long'), ('merchant_lat', 'merchant_long') and unix_time compute transaction speed

In [15]:

temp_df = pd.DataFrame()

# Haversine formula function
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in km
    R = 6371.0

    # Convert degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

temp_df=  data[['unix_time', 'lat', 'long', 'merch_lat', 'merch_long']].to_pandas()
temp_df['tx_duration'] = temp_df['unix_time'].apply(lambda x: x/1e9)
temp_df['distance_km'] = temp_df.apply(
    lambda row: haversine(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)
data['speed'] =  (temp_df['distance_km']/temp_df['tx_duration'])
del temp_df

#### Using variables for makes code cleaner

In [16]:

COL_CARD = 'cc_num'
COL_MCC = 'category'
COL_MERCHANT = 'merchant'
COL_STATE ='state'
COL_CITY ='city'
COL_ZIP = 'zip'

COL_AMOUNT = 'amt'
COL_CITY_POP = 'city_pop'

COL_FRAUD = 'is_fraud'

COL_TIME = 'time'
COL_DAY = 'day'
COL_MONTH = 'month'
COL_YEAR = 'year'
COL_AGE = 'age'
COL_JOB = 'job'
COL_SPEED = 'speed'

NUMERICAL_COLUMNS = [
    COL_AMOUNT, COL_CITY_POP, COL_TIME, COL_AGE, COL_SPEED,
    'lat', 'long', 'merch_lat', 'merch_long' ]


##### Number of cards per user

In [ ]:
len(data.cc_num.unique()) / len((data['first'] + data['last']).unique())

#### Look into numerical columns

In [ ]:
data[NUMERICAL_COLUMNS].describe()

#### Findings
* 'amt' and 'city_pop' have extreme values or outliers compared to mean and median.

In [ ]:
data[COL_AMOUNT].describe()

##### Plot histogram of the 'amt' field

In [ ]:

import matplotlib.pyplot as plt
kde = gaussian_kde(data[COL_AMOUNT].to_pandas())
x_vals = np.linspace(data[COL_AMOUNT].min(), 2000, 100)
plt.plot(x_vals, kde(x_vals), color='blue')

##### Findings
* very few transactions have higher 'amt' values

In [ ]:
data[COL_CITY_POP].describe()

##### Plot histogram of the 'city_pop' field

In [ ]:
kde = gaussian_kde(data[COL_CITY_POP].to_pandas())
x_vals = np.linspace(data[COL_CITY_POP].min(), 100000, 100)
plt.plot(x_vals, kde(x_vals), color='blue')

##### Findings
* Only a few cities have a population over 40,000

#### Let's look into how the amount differ between fraud and non-fraud transactions

In [ ]:
data[COL_AMOUNT].describe()

In [ ]:
# Fraud transactions
data[COL_AMOUNT][data[COL_FRAUD] == 1].describe()

In [ ]:
# Non-fraud transactions
data[COL_AMOUNT][data[COL_FRAUD] == 0].describe()

In [ ]:
# Non-fraud transactions with high value of amount 
data[COL_AMOUNT] [ (data[COL_FRAUD]==0) & (data[COL_AMOUNT] > 1376)  ].describe()

#### Findings

* Average amount in fraud transactions > 8x the average amount in non-fraud transactions
* Interestingly, many non-fraud transactions have high amount as well.

We need to scale the data, and RobustScaler could be a good choice for it.

#### Number of unique values per nominal columns

In [ ]:
# Check how many unique values for 
for col in [COL_STATE, COL_CITY, COL_ZIP, COL_MERCHANT, COL_MCC, COL_CARD]:
    print(f'#unique values ({col}) = {len(data[col].unique())}')


#### Findings
We can binary encode 'state', 'city', 'zip', 'merchant', 'category', 'cc_num', if the columns have good correlation with targets

#### Take a look into distribution of 'time', 'speed' and 'age' columns


##### Plot histogram of transaction 'speed'

In [ ]:
kde = gaussian_kde(data[COL_SPEED].to_pandas())
x_vals = np.linspace(data[COL_SPEED].min(), data[COL_SPEED].max(), 100)
plt.plot(x_vals, kde(x_vals), color='blue')

##### Plot histogram of 'time'
__NOTE__ Time is captured as number of minutes over the span of a day

In [ ]:
kde = gaussian_kde(data[COL_TIME].to_pandas())
x_vals = np.linspace(data[COL_TIME].min(), data[COL_TIME].max(), 100)
plt.plot(x_vals, kde(x_vals), color='blue')

##### Plot histogram of 'age'

In [ ]:
kde = gaussian_kde(data[COL_AGE].to_pandas())
x_vals = np.linspace(data[COL_AGE].min(), data[COL_AGE].max(), 100)
plt.plot(x_vals, kde(x_vals), color='blue')

##### Findings
* It's not obvious from the histogram of 'time,' 'speed,' and 'age' whether they are clear indicators for labeling a transaction as fraud.

#### Define a function to compute correlation of different categorical fields with target

In [31]:
# https://en.wikipedia.org/wiki/Cram%C3%A9r's_V

def cramers_v(x, y):
    confusion_matrix = cudf.crosstab(x, y).to_numpy()
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k-1, r-1))))

##### Compute correlation of different field with target

In [ ]:
sparse_factor = 1
columns_to_compute_corr =  [
    COL_CARD, COL_STATE, COL_CITY, COL_ZIP, COL_MCC, COL_MERCHANT,
    COL_DAY, COL_MONTH, COL_YEAR, COL_JOB, 'gender']
for c1 in columns_to_compute_corr:
    for c2 in [COL_FRAUD]:
        coff =  100 * cramers_v(data[c1][::sparse_factor], data[c2][::sparse_factor])
        print('Correlation ({}, {}) = {:6.2f}%'.format(c1, c2, coff))

#### Findings
* 'day', 'month', and 'year' 'gender' are not important to predict if a transaction is fraud or not

#### Check how City, State and Zip are correlated

In [ ]:

sparse_factor = 1
columns_to_compute_corr =  [COL_STATE, COL_CITY, COL_ZIP]
for c1 in columns_to_compute_corr:
    for c2 in columns_to_compute_corr:
        if c1 not in c2:
            coff =  100 * cramers_v(data[c1][::sparse_factor], data[c2][::sparse_factor])
            print('{} {} {:6.2f}%'.format(c1, c2, coff))

#### Findings
* if we use 'zip' to predict if a transaction is fraud or not, we don't need to use 'city' and 'state'

### Correlation of target with numerical columns

In [ ]:
# https://en.wikipedia.org/wiki/Point-biserial_correlation_coefficient
# Use Point-biserial correlation coefficient(rpb) to check if the numerical columns are important to predict if a transaction is fraud

for col in NUMERICAL_COLUMNS:
    r_pb, p_value = pointbiserialr(data[COL_FRAUD].to_pandas(), data[col].to_pandas())
    print('r_pb ({}) = {:3.2f} with p_value {:3.2f}'.format(col,  r_pb, p_value))

#### Findings
* 'amt' column has positive correlation with target
* other columns, such as 'city_pop', 'time', 'age', 'lat', 'long', 'merch_lat', and  'merch_long' has negligible correlation with target
* Speed can't be ignored as the p_value > 0.05

#### Based on correlation values, select a set of columns (aka fields) to predict whether a transaction is fraud

In [35]:

numerical_predictors = [COL_AMOUNT, COL_SPEED, COL_AGE]
nominal_predictors = [COL_CARD, COL_ZIP, COL_MCC, COL_MERCHANT, COL_JOB]

predictor_columns = numerical_predictors + nominal_predictors

target_column = [COL_FRAUD]

#### Remove duplicates non-fraud data points

In [ ]:
# Remove duplicates data points
fraud_data = data[data[COL_FRAUD] == 1]
data = data[data[COL_FRAUD] == 0]
data = data.drop_duplicates(subset=nominal_predictors)
data = cudf.concat([data, fraud_data])

100*data[COL_FRAUD].value_counts()/len(data)

In [ ]:
# Portion of fraud and non-fraud cases
data[COL_YEAR].value_counts()/len(data)

### Split data
All the transactions were made in year 2019. Let's split the data into three groups based on event month
* Training   - all transactions in 2019
* Validation - all transactions between January and May in 2020
* Test.      - all transactions after May 2020

In [ ]:
if under_sample:    
    fraud_df = data[data[COL_FRAUD]==1]
    non_fraud_df = data[data[COL_FRAUD]==0]
    nr_non_fraud_samples = min((len(data) - len(fraud_df)), int(len(fraud_df)/fraud_ratio))
    data = cudf.concat([fraud_df, non_fraud_df.sample(nr_non_fraud_samples)])

training_idx = data[COL_YEAR] == 2019
validation_idx = (data[COL_YEAR] == 2020)  & (data[COL_MONTH] < 4 )
test_idx = (data[COL_YEAR] == 2020)  & (data[COL_MONTH] >= 4 )

data[COL_FRAUD].value_counts()

In [ ]:
# portion of data for training, test and validation
training_idx.sum()/len(data), validation_idx.sum()/len(data), test_idx.sum()/len(data)

### Scale numerical columns and encode categorical columns of training data

In [40]:
# As some of the encoder we want to use is not available in cuml yet, we can use pandas for now.
# Move training data to pandas for preprocessing
pdf_training = data[training_idx].to_pandas()[predictor_columns + target_column]

In [41]:
#Use binary encoding for categorical columns
columns_for_binary_encoding = nominal_predictors

In [42]:
# Mark categorical column as "category"
pdf_training[nominal_predictors] = pdf_training[nominal_predictors].astype("category")

In [43]:
# encoders to encode categorical columns and scalers to scale numerical columns

bin_encoder = Pipeline(
    steps=[
        ("binary", BinaryEncoder(handle_missing='value', handle_unknown='value'))
    ]
)
onehot_encoder = Pipeline(
    steps=[
        ("onehot", OneHotEncoder())
    ]
)

std_scaler = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("standard", StandardScaler())],
)

robust_scaler = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("robust", RobustScaler())],
)

In [44]:
# compose encoders and scalers in a column transformer
transformer = ColumnTransformer(
    transformers=[
        ("binary", bin_encoder, columns_for_binary_encoding ),    
        ("robust", robust_scaler, [COL_AMOUNT]),
        ("stdscaler", std_scaler, [COL_SPEED, COL_AGE]),
    ], remainder="passthrough"
)

##### Fit column transformer with training data

In [45]:
# Fit column transformer with training data

pd.set_option('future.no_silent_downcasting', True)
transformer = transformer.fit(pdf_training[predictor_columns])

In [46]:
# transformed column names
columns_of_transformed_data = list(
    map(lambda name: name.split('__')[1],
        list(transformer.get_feature_names_out(predictor_columns))))

In [47]:
# data type of transformed columns 
type_mapping = {}
for col in columns_of_transformed_data:
    if col.split('_')[0] in nominal_predictors:
        type_mapping[col] = 'int8'
    elif col in numerical_predictors:
        type_mapping[col] = 'float'
    elif col in target_column:
        type_mapping[col] = data.dtypes.to_dict()[col]

In [48]:
# transform training data
preprocessed_training_data = transformer.transform(pdf_training[predictor_columns])

# Convert transformed data to panda DataFrame
preprocessed_training_data = pd.DataFrame(
    preprocessed_training_data, columns=columns_of_transformed_data)
# Copy target column
preprocessed_training_data[COL_FRAUD] = pdf_training[COL_FRAUD].values
preprocessed_training_data = preprocessed_training_data.astype(type_mapping)

In [49]:
# Save the transformer 

with open(os.path.join(sparkov_base_path, 'preprocessor.pkl'),'wb') as f:
    pickle.dump(transformer, f)

#### Save transformed training data for XGBoost training

In [50]:
with open(os.path.join(sparkov_base_path, 'preprocessor.pkl'),'rb') as f:
    loaded_transformer = pickle.load(f)

In [51]:
# Transform test data using the transformer fitted on training data
pdf_test = data[test_idx].to_pandas()[predictor_columns + target_column]
pdf_test[nominal_predictors] = pdf_test[nominal_predictors].astype("category")

preprocessed_test_data = loaded_transformer.transform(pdf_test[predictor_columns])
preprocessed_test_data = pd.DataFrame(preprocessed_test_data, columns=columns_of_transformed_data)

# Copy target column
preprocessed_test_data[COL_FRAUD] = pdf_test[COL_FRAUD].values
preprocessed_test_data = preprocessed_test_data.astype(type_mapping)

In [52]:
# Transform validation data using the transformer fitted on training data
pdf_validation = data[validation_idx].to_pandas()[predictor_columns + target_column]
pdf_validation[nominal_predictors] = pdf_validation[nominal_predictors].astype("category")

preprocessed_validation_data = loaded_transformer.transform(pdf_validation[predictor_columns])
preprocessed_validation_data = pd.DataFrame(preprocessed_validation_data, columns=columns_of_transformed_data)

# Copy target column
preprocessed_validation_data[COL_FRAUD] = pdf_validation[COL_FRAUD].values
preprocessed_validation_data = preprocessed_validation_data.astype(type_mapping)

## Write out the data for XGB

In [53]:
## Training data
out_path = os.path.join(sparkov_xgb, 'training.csv')
if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
preprocessed_training_data.to_csv(
  out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_training_data.to_parquet(out_path, index=False, compression='gzip')

In [54]:
## validation data
out_path = os.path.join(sparkov_xgb, 'validation.csv')
if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
preprocessed_validation_data.to_csv(
  out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_validation_data.to_parquet(out_path, index=False, compression='gzip')

In [55]:
## test data
out_path = os.path.join(sparkov_xgb, 'test.csv')
preprocessed_test_data.to_csv(
    out_path, header=True, index=False, columns=columns_of_transformed_data + target_column)
# preprocessed_test_data.to_parquet(out_path, index=False, compression='gzip')

In [56]:
# Write untransformed test data that has only (renamed) predictor columns and target
out_path = os.path.join(sparkov_xgb, 'untransformed_test.csv')
pdf_test.to_csv(out_path, header=True, index=False)

In [57]:
# Delete dataFrames that are not needed anymore
del(pdf_training)
del(pdf_validation)
del(pdf_test)
del(preprocessed_training_data)
del(preprocessed_validation_data)
del(preprocessed_test_data)

### GNN Data

#### Setting Vertex IDs
In order to create a graph, the different vertices need to be assigned unique vertex IDs. Additionally, the IDs needs to be consecutive and positive.

There are three nodes groups here: Transactions, Users, and Merchants. 

These IDs are not used in training, just used for graph processing.

In [58]:
# Use the same training data as used for XGBoost
data = data[training_idx]

In [59]:
# a lot of process has occurred, sort the data and reset the index
data = data.sort_values(by=[COL_YEAR, COL_MONTH, COL_DAY, COL_TIME], ascending=False)
data.reset_index(inplace=True, drop=True)

In [60]:
# Each transaction gets a unique ID
COL_TRANSACTION_ID = 'Tx_ID'
COL_MERCHANT_ID = 'Merchant_ID'
COL_USER_ID = 'User_ID'

# The number of transaction is the same as the size of the list, and hence the index value
data[COL_TRANSACTION_ID] = data.index

In [61]:
# Get the max transaction ID to compute first merchant ID
max_tx_id = data[COL_TRANSACTION_ID].max()

In [ ]:
# Convert Merchant string to consecutive integers
merchant_name_to_id = dict((v, k) for k, v in data[COL_MERCHANT].unique().to_dict().items())
data[COL_MERCHANT_ID] = data[COL_MERCHANT].map(merchant_name_to_id) + (max_tx_id + 1)
data[COL_MERCHANT_ID].min(), data[COL_MERCHANT_ID].max()

In [63]:
# Again, get the max merchant ID to compute first user ID
max_merchant_id = data[COL_MERCHANT_ID].max()

In [ ]:
# Convert Card to consecutive user IDs
user_id_to_consecutive_ids = dict((v, k) for k, v in data[COL_CARD].unique().to_dict().items())
data[COL_USER_ID] = data[COL_CARD].map(user_id_to_consecutive_ids) + max_merchant_id + 1
data[COL_USER_ID].min(), data[COL_USER_ID].max()


In [65]:
# Save the max user ID
max_user_id = data[COL_USER_ID].max()

In [ ]:
# Check the the transaction, merchant and user ids are consecutive
id_range = data[COL_TRANSACTION_ID].min(), data[COL_TRANSACTION_ID].max()
print(f'Transaction ID range {id_range}')
id_range = data[COL_MERCHANT_ID].min(), data[COL_MERCHANT_ID].max()
print(f'Merchant ID range {id_range}')
id_range = data[COL_USER_ID].min(), data[COL_USER_ID].max()
print(f'User ID range {id_range}')

In [67]:
# Sanity checks
assert( data[COL_TRANSACTION_ID].max() == data[COL_MERCHANT_ID].min() - 1)
assert( data[COL_MERCHANT_ID].max() == data[COL_USER_ID].min() - 1)
assert(len(data[COL_USER_ID].unique()) == (data[COL_USER_ID].max() - data[COL_USER_ID].min() + 1))
assert(len(data[COL_MERCHANT_ID].unique()) == (data[COL_MERCHANT_ID].max() - data[COL_MERCHANT_ID].min() + 1))
assert(len(data[COL_TRANSACTION_ID].unique()) == (data[COL_TRANSACTION_ID].max() - data[COL_TRANSACTION_ID].min() + 1))

### Write out the data for GNN

#### Create the Graph Edge Data file 
The file is in COO format

In [68]:
COL_GRAPH_SRC = 'src'
COL_GRAPH_DST = 'dst'
COL_GRAPH_WEIGHT = 'wgt'

# User to Transactions
U_2_T = cudf.DataFrame()
U_2_T[COL_GRAPH_SRC] = data[COL_USER_ID]
U_2_T[COL_GRAPH_DST] = data[COL_TRANSACTION_ID]
if make_undirected:
  T_2_U = cudf.DataFrame()
  T_2_U[COL_GRAPH_SRC] = data[COL_TRANSACTION_ID]
  T_2_U[COL_GRAPH_DST] = data[COL_USER_ID]
  U_2_T = cudf.concat([U_2_T, T_2_U])
  del T_2_U


In [69]:
# Transactions to Merchants
T_2_M = cudf.DataFrame()
T_2_M[COL_GRAPH_SRC] = data[COL_MERCHANT_ID]
T_2_M[COL_GRAPH_DST] = data[COL_TRANSACTION_ID]

if make_undirected:
  M_2_T = cudf.DataFrame()
  M_2_T[COL_GRAPH_SRC] = data[COL_TRANSACTION_ID]
  M_2_T[COL_GRAPH_DST] = data[COL_MERCHANT_ID]
  T_2_M = cudf.concat([T_2_M, M_2_T])
  del M_2_T

In [ ]:
Edge = cudf.concat([U_2_T, T_2_M])
Edge[COL_GRAPH_WEIGHT] = 0.0
len(Edge)

In [71]:
# now write out the data
out_path = os.path.join (sparkov_gnn, 'edges.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
  
Edge.to_csv(out_path, header=False, index=False)

In [72]:
del(Edge)
del(U_2_T)
del(T_2_M)

### Now the feature data
Feature data needs to be is sorted in order, where the row index corresponds to the node ID

The data is comprised of three sets of features
* Transactions
* Users
* Merchants

#### To get feature vectors of Transaction nodes, transform the training data using pre-fitted transformer

In [73]:
node_feature_df = pd.DataFrame(
    loaded_transformer.transform(
        data[predictor_columns].to_pandas()
        ),
    columns=columns_of_transformed_data).astype(type_mapping)

node_feature_df[COL_FRAUD] = data[COL_FRAUD].to_pandas()

#### For graph nodes associated with merchant and user, add feature vectors of zeros

In [74]:
# Number of graph nodes for users and merchants 
nr_users_and_merchant_nodes = max_user_id - max_tx_id

In [75]:
if not spread_features:
    # Create feature vector of all zeros for each user and merchant node
    empty_feature_df = cudf.DataFrame(
        columns=columns_of_transformed_data + target_column,
        dtype='int8', 
        index=range(nr_users_and_merchant_nodes)
    )
    empty_feature_df = empty_feature_df.fillna(0)
    empty_feature_df=empty_feature_df.astype(type_mapping)


In [76]:
if not spread_features:
    # Concatenate transaction features followed by features for merchants and user nodes
    node_feature_df = pd.concat([node_feature_df, empty_feature_df.to_pandas()]).astype(type_mapping)

In [77]:
# User specific columns
if spread_features:
    user_specific_columns = [COL_CARD]
    user_specific_columns_of_transformed_data = []

    for col in node_feature_df.columns:
        if '_'.join(col.split('_')[:-1]) in user_specific_columns:
            user_specific_columns_of_transformed_data.append(col)


In [78]:
# Merchant specific columns
if spread_features:
    merchant_specific_columns = [COL_MERCHANT, COL_CITY, COL_ZIP, COL_MCC]
    merchant_specific_columns_of_transformed_data = []
    
    for col in node_feature_df.columns:
        if col.split('_')[0] in merchant_specific_columns:
            merchant_specific_columns_of_transformed_data.append(col)


In [79]:
# Transaction specific columns
if spread_features:
    transaction_specific_columns = list(
        set(numerical_predictors).union(nominal_predictors)
        - set(user_specific_columns).union(merchant_specific_columns))
    transaction_specific_columns_of_transformed_data = []
    
    for col in node_feature_df.columns:
        if col.split('_')[0] in transaction_specific_columns:
            transaction_specific_columns_of_transformed_data.append(col)    

#### Construct feature vector for merchants

In [80]:
if spread_features:
    # Find indices of unique merchants
    idx_df = cudf.DataFrame()
    idx_df[COL_MERCHANT_ID] =  data[COL_MERCHANT_ID]
    idx_df = idx_df.sort_values(by=COL_MERCHANT_ID)
    idx_df = idx_df.drop_duplicates(subset=COL_MERCHANT_ID)
    assert((data.iloc[idx_df.index][COL_MERCHANT_ID] == idx_df[COL_MERCHANT_ID]).all())

In [81]:
if spread_features:
    # Copy merchant specific columns, and set the rest to zero
    merchant_specific_feature_df = node_feature_df.iloc[idx_df.index.to_numpy()]
    merchant_specific_feature_df.\
    loc[:, 
        transaction_specific_columns_of_transformed_data +
          user_specific_columns_of_transformed_data] = 0.0


In [82]:
if spread_features:
    # Find indices of unique users
    idx_df = cudf.DataFrame()
    idx_df[COL_USER_ID] = data[COL_USER_ID]
    idx_df = idx_df.sort_values(by=COL_USER_ID)
    idx_df = idx_df.drop_duplicates(subset=COL_USER_ID)
    assert((data.iloc[idx_df.index][COL_USER_ID] == idx_df[COL_USER_ID]).all())

In [83]:
if spread_features:
    # Copy user specific columns, and set the rest to zero
    user_specific_feature_df = node_feature_df.iloc[idx_df.index.to_numpy()]
    user_specific_feature_df.\
    loc[:,
        transaction_specific_columns_of_transformed_data +
          merchant_specific_columns_of_transformed_data] = 0.0 

In [84]:
# Concatenate features of node, user and merchant
if spread_features:
    
    node_feature_df[merchant_specific_columns_of_transformed_data] = 0.0
    node_feature_df[user_specific_columns_of_transformed_data] = 0.0
    node_feature_df = pd.concat(
        [node_feature_df, merchant_specific_feature_df, user_specific_feature_df]
        ).astype(type_mapping)
    
    # features to save
    node_feature_df = node_feature_df[
        transaction_specific_columns_of_transformed_data +
        merchant_specific_columns_of_transformed_data +
        user_specific_columns_of_transformed_data + [COL_FRAUD]]


In [ ]:
node_feature_df.columns

In [86]:

# target labels to save
label_df = node_feature_df[[COL_FRAUD]]

In [87]:
# Remove target label from feature vectors
_ = node_feature_df.drop(columns=[COL_FRAUD], inplace=True)

#### Write out node features and target labels

In [88]:
# Write node target label to csv file
out_path = os.path.join(sparkov_gnn, 'labels.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))

label_df.to_csv(out_path, header=False, index=False)
# label_df.to_parquet(out_path, index=False, compression='gzip')

In [89]:
# Write node features to csv file
out_path = os.path.join(sparkov_gnn, 'features.csv')

if not os.path.exists(os.path.dirname(out_path)):
  os.makedirs(os.path.dirname(out_path))
node_feature_df[columns_of_transformed_data].to_csv(out_path, header=True, index=False)
# node_feature_df.to_parquet(out_path, index=False, compression='gzip')

In [90]:
# Delete dataFrames
del data
del node_feature_df
del label_df

if spread_features:
    del merchant_specific_feature_df
    del user_specific_feature_df
else:
    del empty_feature_df

#### Number of transaction nodes in training data

In [ ]:
# Number of transaction nodes, needed for GNN training
nr_transaction_nodes = max_tx_id + 1
nr_transaction_nodes

#### Save variable for training and inference

In [92]:

variables_to_save = {
    k: v for k, v in globals().items() if isinstance(v, (str, int)) and k.startswith('COL_')}

In [93]:
variables_to_save['NUM_TRANSACTION_NODES'] = int(nr_transaction_nodes)

In [94]:
# Save the dictionary to a JSON file
with open(os.path.join(sparkov_base_path, 'variables.json'), 'w') as json_file:
    json.dump(variables_to_save, json_file, indent=4)

## That's it!
The data is now ready for processing

## Copyright and License
<hr/>
Copyright (c) 2024, NVIDIA CORPORATION. All rights reserved.

<br/>

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at
 
 http://www.apache.org/licenses/LICENSE-2.0
 
 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.